In [1]:
import numpy as np
from pathlib import Path
import bw2data as bd
import bw2calc as bc
import stats_arrays as sa
import scipy.stats as stats
from gsa_framework.utils import read_pickle, write_pickle
from gsa_framework.models.life_cycle_assessment import LCAModel
from dev.utils_graph_traversal import filter_uncertain_technosphere_exchanges

from dev.utils_local_sa import get_bio_params_local_sa, get_cf_params_local_sa, get_tech_params_local_sa

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
if __name__ == "__main__":

#     path_base = Path(
#         "/Users/akim/PycharmProjects/gsa-framework-master/dev/write_files/"
#     )
    path_base = Path('/data/user/kim_a/protocol_gsa')
    write_dir = path_base 
    write_dir_sct = write_dir / "supply_chain_paper3"
    write_dir_sct.mkdir(exist_ok=True,parents=True)

    bd.projects.set_current("GSA for protocol")
    co = bd.Database("CH consumption 1.0")
    demand_act = [act for act in co if "Food" in act["name"]]
    assert len(demand_act) == 1
    demand_act = demand_act[0]
    demand = {demand_act: 1}
    uncertain_method = ("IPCC 2013", "climate change", "GWP 100a", "uncertain")
    lca = bc.LCA(demand, uncertain_method)
    lca.lci()
    lca.lcia()
    print(lca.score)
    
    # Technosphere
    max_calc = 1e+14
    cutoff = 1e-14
    tech_params_sct_filename = "tech_params_cutoff{:1.0e}_maxcalc{:1.0e}.pickle".format(cutoff, int(max_calc))
    tech_params_sct_filepath = write_dir_sct / tech_params_sct_filename
    if not tech_params_sct_filepath.exists():
        tech_params_sct = filter_uncertain_technosphere_exchanges(lca, cutoff=cutoff, max_calc=max_calc)
        write_pickle(tech_params_sct, tech_params_sct_filepath)
    else:
        tech_params_sct = read_pickle(tech_params_sct_filepath)

211.57670115973556


In [3]:
def add_uncertainty_measure(params, q_range=0.95):
    """Generate uncertainty measures for all params."""

    from stats_arrays import MCRandomNumberGenerator

    mc = MCRandomNumberGenerator(params)
    params = params[mc.ordering]
    
    dt = np.dtype([
        ('row', '<u4'), 
        ('col', '<u4'), 
        ('std2mean', '<f4'), 
        ('std2mean_maxnormalized', '<f4'), 
#         ('var2mean', '<f4'), 
#         ('range2mean', '<f4'), 
        ('uncertainty_type', 'u1'),
        ('contribution', '<f4'),
        ('contribution_maxnormalized', '<f4'),
    ])

    params_uncertainty_measure = np.zeros(len(params), dtype=dt)
    
    q_low = (1-q_range)/2
    q_high = q_low + q_range

    offset = 0
    for uncertainty_type in mc.choices:
        numparams = mc.positions[uncertainty_type]
        if not numparams:
            continue
        current_params = params[offset:numparams + offset]
        params_uncertainty_measure[offset:numparams + offset]['row'] = current_params['row']
        params_uncertainty_measure[offset:numparams + offset]['col'] = current_params['col']
        params_uncertainty_measure[offset:numparams + offset]['uncertainty_type'] = \
            current_params['uncertainty_type']
        if uncertainty_type == sa.LognormalUncertainty:
            s_normal = current_params['scale']
            m_normal = current_params['loc']
            s = np.sqrt( (np.exp(s_normal**2)-1) * np.exp(2*m_normal + s_normal**2) )
            s = s_normal
            m = np.exp(m_normal + (s_normal**2)/2)
            range_ = stats.lognorm.ppf(q_high,s=s_normal, scale=np.exp(m_normal)) - \
                     stats.lognorm.ppf(q_low, s=s_normal, scale=np.exp(m_normal))
        elif uncertainty_type == sa.NormalUncertainty:
            s = current_params['scale']
            m = current_params['loc']
            range_ = stats.norm.ppf(q_high, loc=m, scale=s) - stats.norm.ppf(q_low, loc=m, scale=s)
        elif uncertainty_type == sa.UniformUncertainty:
            min_ = current_params['minimum']
            max_ = current_params['maximum']
            diff = max_ - min_
            s = diff / np.sqrt(12)
            m = current_params['loc']
            range_ = stats.uniform.ppf(q_high, loc=min_, scale=diff) - stats.uniform.ppf(q_low, loc=min_, scale=diff)
        elif uncertainty_type == sa.TriangularUncertainty:
            a = current_params['minimum']
            b = current_params['maximum']
            c = current_params['loc']
            diff = b - a
            m = (a+b+c)/3
            s = np.sqrt((a**2 + b**2 + c**2 - a*b - b*c - a*c)/18)
            range_ = stats.triang.ppf(q_high, c=(c-a)/diff, loc=a, scale=diff) - \
                     stats.triang.ppf(q_low,  c=(c-a)/diff, loc=a, scale=diff)
        else:
            offset += numparams
            continue

        params_uncertainty_measure[offset:numparams + offset]['std2mean'] = np.abs(s/m)
#         params_uncertainty_measure[offset:numparams + offset]['var2mean'] = np.abs((s**2)/m)
#         params_uncertainty_measure[offset:numparams + offset]['range2mean'] = np.abs(range_/m)

        offset += numparams

    params_uncertainty_measure = params_uncertainty_measure[np.argsort(mc.ordering)]
    
    return params_uncertainty_measure

In [4]:
dt = np.dtype([
    ('row', '<u4'), 
    ('col', '<u4'),
])

params_row_col = np.zeros(len(lca.tech_params), dt)
params_row_col[['row', 'col']] = [(p['row'], p['col']) for p in lca.tech_params]

tech_params_sct_row_col = np.zeros(len(tech_params_sct), dt)
tech_params_sct_row_col[['row', 'col']] = [(p[0], p[1]) for p in tech_params_sct]

where_inds = np.where(np.in1d(params_row_col, tech_params_sct_row_col))[0]
input_params = lca.tech_params[where_inds]

params_uncertain = add_uncertainty_measure(input_params)
order = np.argsort(params_uncertain['std2mean'])[-1::-1]

# num_params = -1
selected_tech_params = params_uncertain[order]

tech_params_sct_dict = {(p[0], p[1]): p[2] for p in tech_params_sct}
for p in selected_tech_params:
    p['contribution'] = tech_params_sct_dict[(p['row'], p['col'])]
selected_tech_params['std2mean_maxnormalized'] = \
    selected_tech_params['std2mean'] / max(selected_tech_params['std2mean'])
selected_tech_params['contribution_maxnormalized'] = \
    selected_tech_params['contribution'] / max(selected_tech_params['contribution'])

uncertainty_measure = 0*selected_tech_params['std2mean'] + 1*selected_tech_params['contribution']

num_params = 200
selected_tech_params_sorted = selected_tech_params[np.argsort(uncertainty_measure)[-1::-1][:num_params]]

where_selected_tech = where_inds[order][np.argsort(uncertainty_measure)[-1::-1][:num_params]]
where_selected_tech.sort()

# Local SA

In [5]:
%%time 

const_factors = [1/10, 10]

# Technosphere
tech_params_c = get_tech_params_local_sa(
    where_selected_tech, 
    lca, 
    write_dir_sct, 
    const_factors, 
    tag="sct{}".format(num_params),
)

# Biosphere    
bio_params_c = get_bio_params_local_sa(lca, write_dir_sct, const_factors)

# Characterization
cf_params_c = get_cf_params_local_sa(lca, write_dir_sct, const_factors)

CPU times: user 25min 11s, sys: 47.6 s, total: 25min 58s
Wall time: 2min 37s


# Validation

In [6]:
tech_params_c

{21098: {'input': ('ecoinvent 3.7.1 cutoff',
   'ff04e72f0621a2684d4da40f3a07658d'),
  'output': ('ecoinvent 3.7.1 cutoff', 'b98c7b2cc3bfa5d3aac52040013a4b93'),
  'scores': array([208.58131681, 241.53054462])},
 21101: {'input': ('ecoinvent 3.7.1 cutoff',
   'ff04e72f0621a2684d4da40f3a07658d'),
  'output': ('ecoinvent 3.7.1 cutoff', 'b90a6e04c5884bf9014f3c8efe5300af'),
  'scores': array([208.43303649, 243.01334788])},
 21104: {'input': ('ecoinvent 3.7.1 cutoff',
   'ff04e72f0621a2684d4da40f3a07658d'),
  'output': ('ecoinvent 3.7.1 cutoff', '72ec169c3addd2e5fc3661c256c0c93a'),
  'scores': array([211.19255742, 216.18238655])},
 21291: {'input': ('ecoinvent 3.7.1 cutoff',
   '2cb1c4859a7f7c51144c1aefff2e41a7'),
  'output': ('ecoinvent 3.7.1 cutoff', 'd914569cfeb9d964ad05991cec83f52b'),
  'scores': array([210.95063154, 217.90838843])},
 22287: {'input': ('ecoinvent 3.7.1 cutoff',
   '8105136dfbc27d65cd74eb6482b24b3d'),
  'output': ('ecoinvent 3.7.1 cutoff', '0d50bed0ce9f5cc81ac2e75b0e72f32

In [ ]:
num_params = 2000
rdict = lca.reverse_dict()[0]
selected_tech_params_sorted = selected_tech_params[np.argsort(uncertainty_measure)[-1::-1][:num_params]]
acts_in = []
acts_out = []
for i,p in enumerate(selected_tech_params_sorted):
    act_in  = bd.get_activity(rdict[p['row']])
    act_out = bd.get_activity(rdict[p['col']])
    acts_in.append(act_in['name'] + act_in['location'])
    acts_out.append(act_out['name'] + act_out['location'])
    print("{} FROM: {}, {}".format(i, act_in['name'], act_in['location']))
    print("    TO:   {}, {}".format(act_out['name'], act_out['location']))
    print("std2mean={:5.4f}, score={:5.4f} \n".format(p['std2mean'], p['contribution']))

In [ ]:
sum(selected_tech_params['contribution'])

In [ ]:
len(set(acts_in))

In [ ]:
selected_tech_params_sorted[4]

In [ ]:
bd.get_activity(rdict[18965])

In [ ]:
# num_params = 50000

# fig = make_subplots(specs=[[{"secondary_y": True}]])
# fig.add_trace(
#     go.Scatter(
#         x=np.arange(num_params),
#         y=selected_tech_params['std2mean'][:num_params],
#         mode="markers",
#         name='std2mean',
#         showlegend=True,
#     ),
#     secondary_y=False,
# )

# fig.add_trace(
#     go.Scatter(
#         x=np.arange(num_params),
#         y=selected_tech_params['contribution'][:num_params],
#         mode="markers",
#         name='contribution',
#         showlegend=True,
#     ),
#     secondary_y=True,
# )
# fig

# fig.update_layout(
# width=1000,
# height=800,
# )
# fig.show()

In [ ]:
ei = bd.Database("ecoinvent 3.7.1 cutoff")
a = [act for act in ei if "electricity voltage transformation from high to medium" in act['name'] and 'CN-SGCC' in act['location']][0]

In [ ]:
e = list(a.exchanges())[1]

In [ ]:
list(a.exchanges())

In [ ]:
e.as_dict()

In [ ]:
input_params[order][:30]

In [ ]:
q_range = 0.95
q_low = (1-q_range)/2
q_high = q_low + q_range
    
s_normal = 0.4
m_normal = 26.148415
s = np.sqrt( (np.exp(s_normal**2)-1) * np.exp(2*m_normal + s_normal**2) )
m = np.exp(m_normal + (s_normal**2)/2)
range_ = ss.lognorm.ppf(q_high,s=s_normal, scale=np.exp(m_normal)) - \
         ss.lognorm.ppf(q_low, s=s_normal, scale=np.exp(m_normal))

In [ ]:
s/m, (s**2)/m, range_ / m

In [ ]:
497270804945.37006  -  103664898722.04993

In [ ]:
range_